In [ ]:
import os
import re
import shutil

In [ ]:
from utils import cbar_dicts, echart_dicts

In [ ]:
# update echarts to 5.4 to get legend to work
from panel.models import echarts
f_echarts = echarts.__dict__['__file__']

In [ ]:
import glob
import pandas as pd
import geopandas as gpd
import pickle
import param as pm
import panel as pn
pn.extension('echarts')
import numpy as np
import xarray as xr
import rioxarray as riox
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
from holoviews import streams
import affine
from bokeh.models.formatters import PrintfTickFormatter
from holoviews.plotting.util import process_cmap
from datetime import datetime, timedelta
from pyproj import Proj, transform
from shapely.geometry import Polygon
from copy import deepcopy
from hlsstack.hls_funcs.masks import shp2mask
from hlsstack.hls_funcs.predict import xr_cdf

css = '''
.bk.panel-widget-box {
  background: #f0f0f0;
  border-radius: 5px;
  border: 1px black solid;
}
.bk.stats-box {
  background: rgba(255, 255, 255, 0.0);
  border-radius: 0px;
  padding: 5px;
  border: 1px black solid;
}
.bk.stats-box-nopad {
  background: rgba(255, 255, 255, 0.0);
  border-radius: 0px;
  padding: 1px;
  border: 1px black solid;
}
'''

pn.extension(raw_css=[css])

pn.param.ParamMethod.loading_indicator = True

In [ ]:
prefix = 'cper'
gcloud = True
browser = True
baseDIR = '/mnt/gcs'
#shp_f = 'data/ground/cper_pastures_2017_dissolved.shp'
shp_f = 'data/ground/cper_pastures_2025_w_carm3_zones_w_excl.shp'
means_f = 'data/gcloud/hls_cper_means.csv'

yr_comp = 2020

In [ ]:
if gcloud:
    os.environ["CPL_MACHINE_IS_GCE"] = "YES"
    cper_f = os.path.join(baseDIR, shp_f)
    aoi_means_f = os.path.join(baseDIR, means_f)
else:
    cper_f = os.path.join(shp_f)
    aoi_means_f = os.path.join(means_f)

In [ ]:
sngl_chunks = {'date': 1, 'y': -1, 'x': -1}
ts_chunks = {'date': -1, 'y': 50, 'x': 50}

In [ ]:
if gcloud:
    yr_list = [os.path.basename(x).split('_')[2] for x in glob.glob(os.path.join(baseDIR, 'data/gcloud/hls_' + prefix + '*_gcloud.nc'))]
    yr = int(yr_list[-1])
    dsDIR = os.path.join(baseDIR, 'data/gcloud/')
else:
    yr_list = [os.path.basename(x).split('_')[2] for x in glob.glob('data/gcloud/hls_' + prefix + '*_gcloud.nc')]
    yr = int(yr_list[-1])
    dsDIR = 'data/gcloud/'

ds = riox.open_rasterio(os.path.join(dsDIR, 'hls_' + prefix + '_' + str(yr) + '_gcloud.nc'),
                            masked=True).chunk(sngl_chunks)
ds['date'] = [datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S') for x in ds['date'].values]
ds['date'] = ds['date'].dt.date
ds = ds.where(ds < ds.attrs['_FillValue'])
ds = ds.where(ds != np.inf)

In [ ]:
if gcloud:
    ds_ndvi_lta = riox.open_rasterio(os.path.join(baseDIR, 'data/ee_lta/' + prefix + '_ee_ndvi_landsat_wkly_lta.nc'),
                                     masked=True).chunk(sngl_chunks)
    ds_bm_lta = riox.open_rasterio(os.path.join(baseDIR, 'data/hls_lmf_lta/' + prefix + '_bm_lta_wkly_2000_2024_gcloud.nc'),
                                     masked=True)
    ds_cp_lta = riox.open_rasterio(os.path.join(baseDIR, 'data/hls_lmf_lta/' + prefix + '_cp_lta_wkly_2000_2024_gcloud.nc'),
                                     masked=True)
else:
    ds_ndvi_lta = riox.open_rasterio('data/ee_lta/' + prefix + '_ee_ndvi_landsat_wkly_lta.nc', masked=True)
    ds_bm_lta = riox.open_rasterio('data/hls_lmf_lta/' + prefix + '_bm_lta_wkly_2000_2024_gcloud.nc', masked=True)
    ds_cp_lta = riox.open_rasterio('data/hls_lmf_lta/' + prefix + '_cp_lta_wkly_2000_2024_gcloud.nc', masked=True)

ds_ndvi_lta['date'] = [datetime.strptime(re.sub('2020', str(yr), str(x)),'%Y-%m-%d %H:%M:%S') for x in ds_ndvi_lta['date'].values]
ds_ndvi_lta['date'] = ds_ndvi_lta['date'].dt.date
ds_ndvi_lta = ds_ndvi_lta.reindex({'y': ds.y, 'x': ds.x}, method='nearest', tolerance=30)#.isnull().all()

ds_bm_lta = ds_bm_lta.rename({'week': 'date'}).chunk(sngl_chunks)
ds_bm_lta['date'] = [datetime(yr, 1, 1) + timedelta(weeks=int(w-1)) for w in ds_bm_lta['date'].values]
ds_bm_lta = ds_bm_lta.reindex({'y': ds.y, 'x': ds.x}, method='nearest', tolerance=30)

ds_cp_lta = ds_cp_lta.rename({'week': 'date'}).chunk(sngl_chunks)
ds_cp_lta['date'] = [datetime(yr, 1, 1) + timedelta(weeks=int(w-1)) for w in ds_cp_lta['date'].values]
ds_cp_lta = ds_cp_lta.reindex({'y': ds.y, 'x': ds.x}, method='nearest', tolerance=30)

In [ ]:
cper = gpd.read_file(cper_f).to_crs(ds_ndvi_lta.rio.crs.to_epsg())
cper = cper.rename(columns={'Viewer_ID': 'Pasture'})
cper_bbox = cper.buffer(500).total_bounds
cper_info = cper[['Pasture', 'geometry']].reset_index(drop=True).reset_index().rename(columns={'index': 'id'})
cper_mask_shp = [(row.geometry, row.id+1) for _, row in cper_info.iterrows()]
cper_mask = shp2mask(shp=cper_mask_shp, 
                     transform=ds.isel(date=-1)['Biomass'].rio.transform(), 
                     outshape=ds.isel(date=-1)['Biomass'].shape, 
                     xr_object=ds.isel(date=-1)['Biomass'])
aoi_means = pd.read_csv(aoi_means_f, parse_dates=['date'])
aoi_means.loc[aoi_means['Year'] != str(yr), 
              'date'] = aoi_means['date'][aoi_means['Year'] != str(yr)].dt.isocalendar().week.transform(
    lambda x: datetime(yr, 1, 1) + timedelta(weeks=x-1))
aoi_means = aoi_means.sort_values(['Year', 'Pasture', 'date'])
aoi_means_yr_list = list(aoi_means['Year'][aoi_means['Year'] != 'long-term avg.'].astype('int').unique()) 

In [ ]:
aoi_means = aoi_means.groupby(['Pasture', 'date', 'Year']).mean().reset_index()
aoi_means['Biomass_raw'] = aoi_means['Biomass_raw'].round(0)
aoi_means['NDVI_raw'] = aoi_means['NDVI_raw'].round(3)
aoi_means = aoi_means.dropna(subset=['Biomass'])

In [ ]:
# get averages for TRM and Heavy
trm_pasts = ['15E', '7NW', '1E', '17N', '19N', '31W', '25SE', '24W', '20SE', '13W']
hvy_pasts = ['1W', '28N', '32W', '23E']

trm_means = aoi_means[aoi_means['Pasture'].isin(trm_pasts)].drop(columns='Pasture').groupby(['date', 'Year']).mean().reset_index()
trm_means['Biomass'] = trm_means['Biomass'].round(0)
trm_means['NDVI'] = trm_means['NDVI'].round(3)
trm_means['Pasture'] = 'TRM'

hvy_means = aoi_means[aoi_means['Pasture'].isin(hvy_pasts)].drop(columns='Pasture').groupby(['date', 'Year']).mean().reset_index()
hvy_means['Biomass'] = hvy_means['Biomass'].round(0)
hvy_means['NDVI'] = hvy_means['NDVI'].round(3)
hvy_means['Pasture'] = 'Heavy'

aoi_means = pd.concat([aoi_means, trm_means, hvy_means])

In [ ]:
from matplotlib.pyplot import get_cmap
past_cmap = get_cmap("cet_glasbey_hv")
cper['linecolor'] = [past_cmap.colors[idx] for idx, x in enumerate(cper['Pasture'])]

In [ ]:
from bokeh.models import WheelZoomTool, HoverTool
wheel_zoom = WheelZoomTool(zoom_on_axis=False)
hover_cust = HoverTool(tooltips=[("", "@Pasture")])
print('prepping app')

In [ ]:
class getData(pm.Parameterized):
    control_width = 200
    stats_width = 380
    stats_margin = (5, 1)
    sngl_chunks = sngl_chunks
    ts_chunks = ts_chunks
    aoi_means = aoi_means
    
    viz_sel = pn.widgets.RadioButtonGroup(options=['Basemap', 
                                                   'Cover',
                                                   'Greenness (NDVI)',
                                                   'Biomass',
                                                   'Crude protein',
                                                   'Bare ground change',
                                                   'Greenness change',
                                                   'Biomass change',
                                                   'Crude protein change',
                                                   'Relative greenness',
                                                   'Relative biomass',
                                                   'Relative CP',
                                                   'Biomass threshold'],
                                          value='Cover',
                                          align='start', 
                                          orientation='vertical',
                                          width=control_width)

    def viz_callback(target, event):
        if event.new == target.name:
            target.value = True
        else:
            target.value = False

    class title_string(pm.Parameterized):
        title = pm.String('CPER-wide')
    
    #lta_viz = pn.widgets.Checkbox(name='Show long-term avg.', value=True)
    
    cov_viz = pn.widgets.Checkbox(name='Cover', value=True)
    bm_viz = pn.widgets.Checkbox(name='Biomass', value=False)
    bm_chng_viz = pn.widgets.Checkbox(name='Biomass change', value=False)
    bm_rel_viz = pn.widgets.Checkbox(name='Relative biomass', value=False)
    thresh_viz = pn.widgets.Checkbox(name='Biomass threshold', value=False)
    ndvi_viz = pn.widgets.Checkbox(name='Greenness (NDVI)', value=False)
    ndvi_rel_viz = pn.widgets.Checkbox(name='Relative greenness', value=False)
    ndvi_chng_viz = pn.widgets.Checkbox(name='Greenness change', value=False)
    bare_chng_viz = pn.widgets.Checkbox(name='Bare ground change', value=False)
    cp_viz = pn.widgets.Checkbox(name='Crude protein', value=False)
    cp_rel_viz = pn.widgets.Checkbox(name='Relative CP', value=False)
    cp_chng_viz = pn.widgets.Checkbox(name='Crude protein change', value=False)

    viz_sel.link(cov_viz, callbacks={'value': viz_callback})
    viz_sel.link(bm_viz, callbacks={'value': viz_callback})
    viz_sel.link(bm_chng_viz, callbacks={'value': viz_callback})
    viz_sel.link(bm_rel_viz, callbacks={'value': viz_callback})
    viz_sel.link(thresh_viz, callbacks={'value': viz_callback})
    viz_sel.link(ndvi_viz, callbacks={'value': viz_callback})
    viz_sel.link(ndvi_rel_viz, callbacks={'value': viz_callback})
    viz_sel.link(ndvi_chng_viz, callbacks={'value': viz_callback})
    viz_sel.link(bare_chng_viz, callbacks={'value': viz_callback})
    viz_sel.link(cp_viz, callbacks={'value': viz_callback})
    viz_sel.link(cp_rel_viz, callbacks={'value': viz_callback})
    viz_sel.link(cp_chng_viz, callbacks={'value': viz_callback})
    
    ds_picker = pn.widgets.Select(name='Select map year', options=yr_list, value=str(yr),
                                  width=control_width)
    year_picker = pn.widgets.Select(name='Comparison year', options=['None'] + aoi_means_yr_list, value='None',
                                   width=control_width)
    
    thresh_picker = pn.widgets.IntSlider(name='Biomass threshold', start=200, end=2000, step=50, value=400,
                                 format=PrintfTickFormatter(format='%d lbs/ac'), width=control_width)
    date_diff_picker = pn.widgets.IntSlider(name='Days', start=-120, end=0, step=5, value=-30,
                                            width=control_width)

    cov_dict = {'R': 'Dry veg',
                'G': 'Green veg',
                'B': 'Bare ground'}

    map_args = dict(rasterize=True, project=False, dynamic=True, xticks=None, yticks=None)
    base_opts = dict(backend='bokeh', xaxis=None, yaxis=None, width=950, height=800,
                         padding=0, active_tools=['pan', wheel_zoom], toolbar='left', title='')
    map_opts = dict(responsive=False, xaxis=None, yaxis=None, width=950, height=800,
                     padding=0, tools=['pan', wheel_zoom, 'box_zoom', 'tap'],
                     active_tools=['pan', wheel_zoom], toolbar='left', title='')
    
    poly_opts = dict(fill_color=['', ''], fill_alpha=[0.0, 0.0], line_color=['#1b9e77', '#d95f02'],
                 line_width=[3, 3])    
    
    bg_col = '#ffffff'
    past_col = echart_dicts.Line().past_col
    poly_col = echart_dicts.Line().poly_col
    
    date = pn.widgets.DatePicker(name='Select date', width=100)
    enabled_dates = [pd.Timestamp(x).to_pydatetime().date() for x in ds['date'].values]
    date.enabled_dates = enabled_dates
      
    tiles = hv.element.tiles.EsriImagery().opts(**base_opts, level='glyph',
                                                                   xlim=(cper_bbox[0], cper_bbox[2]),
                                                                   ylim=(cper_bbox[1], cper_bbox[3]))
    labels = hv.element.tiles.EsriReference().opts(**base_opts, level='overlay')
    
    base_rng = hv.streams.RangeXY(source=tiles,
                                  x_range = (cper_bbox[0], cper_bbox[2]),
                                  y_range = (cper_bbox[1], cper_bbox[3]))
       
    #basemap = hv.element.tiles.EsriImagery().opts(**map_opts, level='glyph')
        
    polys = hv.Polygons([])

    max_polys = 1
    
    poly_stream = streams.PolyDraw(source=polys, drag=True, num_objects=max_polys,
                                    show_vertices=True, styles=poly_opts)    
    
    past_polys = cper.hvplot(geo=True, crs=3857, hover_cols=['Pasture']).opts(tools=[hover_cust, 'tap'],
                                                      fill_color=None,
                                                      cmap=past_cmap, 
                                                      fill_alpha=0.0, 
                                                      line_color='white', 
                                                      line_width=2,
                                                      line_alpha=0.7,
                                                      show_legend=False, 
                                                      hover_line_color='red',
                                                      selection_line_color=past_col,
                                                      selection_line_alpha=1.0,
                                                      nonselection_line_color='grey',
                                                      nonselection_line_alpha=0.8)
    past_sel = hv.streams.Selection1D(source=past_polys)

    cbar_dict = cbar_dicts.CBar(control_width).full_dict
    
    # format biomass gauge
    bm_gauge_colors = np.array(process_cmap(cbar_dict['Biomass']['cmap'], ncolors=8))
    bm_gauge_bounds = (0, 3500)
    bm_gauge_colors_list = list(zip(np.linspace(0, cbar_dict['Biomass']['clim'][1]/bm_gauge_bounds[1], 
                                                len(bm_gauge_colors)-1, endpoint=True),
                                    bm_gauge_colors[:-1]))
    bm_gauge_colors_list[0] = (cbar_dict['Biomass']['clim'][0]/bm_gauge_bounds[1], bm_gauge_colors[0])
    bm_gauge_colors_list.append((1.0, bm_gauge_colors[-1]))
    bm_gauge = pn.indicators.Gauge(
        name='Biomass', bounds=bm_gauge_bounds, format='{value} lbs/ac', tooltip_format='{b} : {c} lbs/ac',
        colors=bm_gauge_colors_list,
        num_splits=5, align='start', title_size=12,
        start_angle=180, end_angle=0,
        height=200, width=stats_width,
        value=0, margin=stats_margin, css_classes=['stats-box'])
    bm_gauge_pasts = deepcopy(bm_gauge)

    # format crude protein gauge
    cp_gauge_colors = np.array(process_cmap(cbar_dict['Crude protein']['cmap'], 
                                            ncolors=2+cbar_dict['Crude protein']['clim'][1]-cbar_dict['Crude protein']['clim'][0]))
    cp_gauge_bounds = (0.0, 14.0)
    cp_gauge_colors_list = list(zip(np.linspace(cbar_dict['Crude protein']['clim'][0]/cp_gauge_bounds[1], 
                                                cbar_dict['Crude protein']['clim'][1]/cp_gauge_bounds[1], 
                                              len(cp_gauge_colors)-1),
                                    cp_gauge_colors[:-1]))
    cp_gauge_colors_list[0] = (cbar_dict['Crude protein']['clim'][0]/cp_gauge_bounds[1], cp_gauge_colors[0])
    cp_gauge_colors_list.append((1.0, cp_gauge_colors[-1]))
    cp_gauge = pn.indicators.LinearGauge(
        name='Crude protein', value=0, bounds=cp_gauge_bounds, format='{value:.2f} %',
        colors=cp_gauge_colors_list,
        value_size='20pt',
        show_boundaries=True, horizontal=True,
        height=stats_width, 
        width=150, 
        margin=stats_margin, 
        css_classes=['stats-box-nopad']
    )
    cp_gauge_pasts = deepcopy(cp_gauge)

    # format biomass histogram bar plot
    bm_cmin = 100
    bm_cmax = 1000
    bm_plotrange = echart_dicts.Bar().bm_plotrange
    bm_colors = np.array(process_cmap('Inferno', ncolors=len(bm_plotrange)))
    bm_colors[bm_plotrange < bm_cmin] = bm_colors[0]
    bm_colors[bm_plotrange > bm_cmax] = bm_colors[-1]
    bm_colors[(bm_plotrange >= bm_cmin) & 
              (bm_plotrange <= bm_cmax)] = process_cmap('Inferno', ncolors=len(np.arange(bm_cmin, bm_cmax+50, 50)))
    bm_hist = echart_dicts.Bar().bm_hist
    bm_echart = pn.pane.ECharts(bm_hist, height=250, width=stats_width, margin=stats_margin, css_classes=['stats-box'])
    bm_hist_pasts = deepcopy(bm_hist)
    bm_echart_pasts = pn.pane.ECharts(bm_hist_pasts, height=250, width=stats_width, margin=stats_margin, css_classes=['stats-box'])

    # format biomass threshold bar plot
    thresh_plotrange = np.arange(0, 1.2, 0.2)
    thresh_colors = process_cmap('Spectral_r', ncolors=len(thresh_plotrange)-1)
    thresh_labels = echart_dicts.Bar().thresh_labels
    thresh_bar = echart_dicts.Bar().thresh_bar
    thresh_bar['xAxis']['name'] = 'Probability of biomass less than ' + str(thresh_picker.value_throttled) + 'lbs/ac'
    thresh_echart = pn.pane.ECharts(thresh_bar, height=250, width=stats_width, margin=stats_margin, css_classes=['stats-box'])
    thresh_bar_pasts = deepcopy(thresh_bar)
    thresh_echart_pasts = pn.pane.ECharts(thresh_bar_pasts, height=250, width=stats_width, margin=stats_margin, css_classes=['stats-box'])
    
    # format cover pie chart
    cov_pie = echart_dicts.Pie().cov_pie
    cov_echart = pn.pane.ECharts(cov_pie, height=250, width=stats_width, margin=stats_margin, css_classes=['stats-box'])
    cov_pie_pasts = deepcopy(cov_pie)
    cov_echart_pasts = pn.pane.ECharts(cov_pie_pasts, height=250, width=stats_width, margin=stats_margin, css_classes=['stats-box'])

    # format biomass time series line chart
    ts_bm = echart_dicts.Line(aoi_means, ds_picker.value, year_picker.value).ts_bm
    ts_bm_echart = pn.pane.ECharts(ts_bm, height=250, width=stats_width, margin=stats_margin, css_classes=['stats-box'])

    # format ndvi time series line chart
    ts_ndvi = echart_dicts.Line(aoi_means, ds_picker.value, year_picker.value).ts_ndvi
    ts_ndvi_echart = pn.pane.ECharts(ts_ndvi, height=250, width=stats_width, margin=stats_margin, css_classes=['stats-box'])

    # format crude protein time series line chart
    ts_cp = echart_dicts.Line(aoi_means, ds_picker.value, year_picker.value).ts_cp
    ts_cp_echart = pn.pane.ECharts(ts_cp, height=250, width=stats_width, margin=stats_margin, css_classes=['stats-box'])

    # format cover time series stacked line chart
    ts_cov_pasts = echart_dicts.Line().ts_cov
    ts_cov_echart_pasts = pn.pane.ECharts(ts_cov_pasts, height=250, width=stats_width, margin=stats_margin, css_classes=['stats-box'])
    ts_cov_poly = deepcopy(ts_cov_pasts)
    ts_cov_poly['title'][0]['text'] = 'Cover (Drawing)'
    ts_cov_echart_poly = pn.pane.ECharts(ts_cov_poly, height=250, width=stats_width, margin=stats_margin, css_classes=['stats-box']) 
    
    
    def __init__(self, **params):
        super(getData, self).__init__(**params)
        if gcloud:
            self.ds = ds.chunk(self.sngl_chunks)
            self.ds_ts = ds.copy().chunk(self.ts_chunks)
            self.ds_ndvi_lta = ds_ndvi_lta.chunk(self.sngl_chunks)
            self.ds_bm_lta = ds_bm_lta.chunk(self.sngl_chunks)
            self.ds_cp_lta = ds_cp_lta.chunk(self.sngl_chunks)
        else:
            self.ds = ds
            self.ds_ts = ds
            self.ds_ndvi_lta = ds_ndvi_lta
            self.ds_bm_lta = ds_bm_lta
            self.ds_cp_lta = ds_cp_lta
        self.aoi_means = aoi_means
        self.ds_sel = None
        self.da_thresh = None

        self.all_maps = None
        self.cov_map = None
        self.bm_map = None
        self.thresh_map = None
        self.cp_map = None
        
        self.cov_stats = None
        self.bm_stats = None
        self.thresh_stats = None
        self.stats_title = self.title_string()
        #self.ts_bm = ts_bm
        
        self.bm_ts_dat_poly = []
        
        self.date.value = pd.to_datetime(self.ds['date'].values[-1]).date()
        
        self.poly_stream.add_subscriber(self.update_stats)
        self.past_sel.add_subscriber(self.update_stats)
        
        self.map_init = self.tiles * self.labels
        
        self.controls = pn.Column(pn.Spacer(height=5, margin=0),
                                  self.viz_sel,
                                  pn.Spacer(height=5, margin=0),
                                  pn.pane.Markdown('  *Map legend*',
                                                   margin=(0, 0, 0, 0)),
                                  self.update_colorbar, 
                                  pn.Spacer(height=5, margin=0),
                                  self.update_slider,                                  
                                  width=int(self.control_width*1.2))
        
        self.view = self._create_view()
    
    def make_colorbar(self, key, orientation = 'horizontal', position = 'top', colorbar_opts = {}, **kwargs):
        ## create an invisible hv.Heatmap plot just to use its colorbar
        cmap = self.cbar_dict[key]['cmap']
        clim = self.cbar_dict[key]['clim']
        barlim = self.cbar_dict[key]['barlim']
        colorbar_opts = self.cbar_dict[key]['colorbar_opts']
        
        hm = hv.HeatMap([(0, 0, barlim[0]), (0, 1, barlim[1])]).opts(hooks=[lambda p, _: p.state.update(border_fill_alpha=0.0)])
        kwargs.update(dict(colorbar=True,
                           colorbar_opts=colorbar_opts,
                           clim=clim,
                           alpha=0.0,
                           show_frame=False,
                           frame_height=0,
                           colorbar_position=position, 
                           toolbar="disable",
                           axiswise=True, framewise=True, shared_axes=False,
                           margin=(0, 0, 0, 0),
                           cmap=cmap))
        return hm.opts(xaxis = None, yaxis = None, **kwargs)
    
    @pm.depends('ds_picker.value', watch=True)
    def update_ds(self):
        new_date = pd.to_datetime('-'.join([str(self.ds_picker.value),
                                            str(self.date.value.month),
                                            str(self.date.value.day)])).date()
        self.ds = riox.open_rasterio(os.path.join(dsDIR, 'hls_' + prefix + '_' + self.ds_picker.value + '_gcloud.nc'),
                            masked=True).chunk(sngl_chunks)
        self.ds['date'] = [datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S') for x in self.ds['date'].values]
        self.ds['date'] = self.ds['date'].dt.date
        self.ds = self.ds.where(self.ds < self.ds.attrs['_FillValue'])
        self.ds = self.ds.where(self.ds != np.inf)
        self.enabled_dates = [pd.Timestamp(x).to_pydatetime().date() for x in self.ds['date'].values]
        self.date.enabled_dates = self.enabled_dates

        # update long-term average data to match date range of other datasets
        self.ds_ndvi_lta['date'] = [pd.to_datetime('-'.join([self.ds_picker.value] + 
                                                            x.split('-')[1:])) for x in self.ds_ndvi_lta['date'].values.astype('str')]
        self.ds_ndvi_lta['date'] = self.ds_ndvi_lta['date'].dt.date
        self.ds_bm_lta['date'] = [pd.to_datetime('-'.join([self.ds_picker.value] + 
                                                            x.split('-')[1:])) for x in self.ds_bm_lta['date'].values.astype('str')]
        self.ds_bm_lta['date'] = self.ds_bm_lta['date'].dt.date
        self.ds_cp_lta['date'] = [pd.to_datetime('-'.join([self.ds_picker.value] + 
                                                            x.split('-')[1:])) for x in self.ds_cp_lta['date'].values.astype('str')]
        self.ds_cp_lta['date'] = self.ds_cp_lta['date'].dt.date
        
        
        if new_date in self.date.enabled_dates:
            self.date.value = pd.to_datetime('-'.join([str(self.ds_picker.value),
                                            str(self.date.value.month),
                                            str(self.date.value.day)])).date()
        else:
            self.date.value = pd.to_datetime(self.ds['date'].values[-1]).date()             
    
    @pm.depends('date.value', 'thresh_picker.value_throttled', 'date_diff_picker.value_throttled')
    def create_maps(self):
        self.ds_sel = self.ds.sel(date=self.date.value).compute()
        if self.ds_sel is not None:
            if self.poly_stream.data is not None:
                self.polys = self.poly_stream.element
                self.poly_stream = streams.PolyDraw(source=self.polys, drag=True, num_objects=self.max_polys,
                               show_vertices=True, styles=self.poly_opts)
            else:
                self.polys = self.polys
            self.poly_stream.add_subscriber(self.update_stats)
            
            # get the nearest long-term average data
            self.ds_ndvi_lta_sel = self.ds_ndvi_lta.sel(date=self.date.value, method='nearest').compute()
            self.ds_bm_lta_sel = self.ds_bm_lta.sel(date=self.date.value, method='nearest').compute()
            self.ds_cp_lta_sel = self.ds_cp_lta.sel(date=self.date.value, method='nearest').compute()
            
            # create ndvi change dataarray
            self.da_ndvi_chng = (self.ds_sel['NDVI'] - self.ds['NDVI'].shift(
                {'date': -1 * self.date_diff_picker.value_throttled})).sel(date=self.date.value).compute()
            
            # create bare ground change dataarray
            self.da_bare_chng = (self.ds_sel['BARE'] - self.ds['BARE'].shift(
                {'date': -1 * self.date_diff_picker.value_throttled})).sel(date=self.date.value).compute()
            
            # create biomass change dataarray
            self.da_bm_chng = (self.ds_sel['Biomass'] - self.ds['Biomass'].shift(
                {'date': -1 * self.date_diff_picker.value_throttled})).sel(date=self.date.value).compute()

            # create crude protein change dataarray
            self.da_cp_chng = (self.ds_sel['CP'] - self.ds['CP'].shift(
                {'date': -1 * self.date_diff_picker.value_throttled})).sel(date=self.date.value).compute()
            
            # create biomass threshold dataarray
            #da_thresh_pre = (np.log(self.thresh_picker.value_throttled) - 
            #                 np.log(self.ds_sel['Biomass'])) / self.ds_sel['Biomass_SE']
            da_thresh_pre = (self.thresh_picker.value_throttled - 
                             self.ds_sel['Biomass']) / self.ds_sel['Biomass_SE']
            self.da_thresh = xr_cdf(da_thresh_pre)
            self.da_thresh.name = 'Threshold'

            self.cov_map = self.ds_sel[['SD', 'GREEN', 'BARE']].to_array().hvplot.rgb(x='x', y='y', 
                                                                                       bands='variable',
                                                                                       **self.map_args).opts(**self.map_opts)
            self.bm_map = self.ds_sel['Biomass'].hvplot(x='x', y='y',
                                       cmap=self.cbar_dict['Biomass']['cmap'],
                                       clim=self.cbar_dict['Biomass']['clim'],
                                       colorbar=False,
                                       **self.map_args).opts(**self.map_opts)
            self.ndvi_chng_map = self.da_ndvi_chng.hvplot(x='x', y='y',
                                       cmap=self.cbar_dict['Greenness change']['cmap'],
                                       clim=self.cbar_dict['Greenness change']['clim'],
                                       colorbar=False,
                                       **self.map_args).opts(**self.map_opts)
            self.bare_chng_map = self.da_bare_chng.hvplot(x='x', y='y',
                                       cmap=self.cbar_dict['Bare ground change']['cmap'],
                                       clim=self.cbar_dict['Bare ground change']['clim'],
                                       colorbar=False,
                                       **self.map_args).opts(**self.map_opts)
            self.bm_chng_map = self.da_bm_chng.hvplot(x='x', y='y',
                                       cmap=self.cbar_dict['Biomass change']['cmap'],
                                       clim=self.cbar_dict['Biomass change']['clim'],
                                       colorbar=False,
                                       **self.map_args).opts(**self.map_opts)
            self.thresh_map = self.da_thresh.hvplot(x='x', y='y',
                                               cmap=self.cbar_dict['Biomass threshold']['cmap'],
                                               clim=self.cbar_dict['Biomass threshold']['clim'], 
                                               colorbar=False,
                                               **self.map_args).options(color_levels=5).opts(**self.map_opts)

            self.ndvi_map = self.ds_sel['NDVI'].hvplot(x='x', y='y',
                                           cmap=self.cbar_dict['Greenness (NDVI)']['cmap'],
                                           clim=self.cbar_dict['Greenness (NDVI)']['clim'],
                                           colorbar=False,
                                           **self.map_args).opts(**self.map_opts)
            
            self.ndvi_rel_map = ((self.ds_sel['NDVI']/self.ds_ndvi_lta_sel)*100).hvplot(x='x', y='y',
                               cmap=self.cbar_dict['Relative greenness']['cmap'],
                               clim=self.cbar_dict['Relative greenness']['clim'],
                               colorbar=False,
                               **self.map_args).opts(**self.map_opts)

            self.bm_rel_map = ((self.ds_sel['Biomass']/self.ds_bm_lta_sel)*100).hvplot(x='x', y='y',
                               cmap=self.cbar_dict['Relative biomass']['cmap'],
                               clim=self.cbar_dict['Relative biomass']['clim'],
                               colorbar=False,
                               **self.map_args).opts(**self.map_opts)
            self.cp_rel_map = ((self.ds_sel['CP']/self.ds_cp_lta_sel)*100).hvplot(x='x', y='y',
                               cmap=self.cbar_dict['Relative CP']['cmap'],
                               clim=self.cbar_dict['Relative CP']['clim'],
                               colorbar=False,
                               **self.map_args).opts(**self.map_opts)
            self.cp_map = self.ds_sel['CP'].hvplot(x='x', y='y',
                                       cmap=self.cbar_dict['Crude protein']['cmap'],
                                       clim=self.cbar_dict['Crude protein']['clim'],
                                       colorbar=False,
                                       **self.map_args).opts(**self.map_opts)
            self.cp_chng_map = self.da_cp_chng.hvplot(x='x', y='y',
                                       cmap=self.cbar_dict['Crude protein change']['cmap'],
                                       clim=self.cbar_dict['Crude protein change']['clim'],
                                       colorbar=False,
                                       **self.map_args).opts(**self.map_opts)

            if self.base_rng.x_range is not None:
                self.all_maps = pn.Row(self.tiles.apply.opts(xlim=self.base_rng.param.x_range,
                                                        ylim=self.base_rng.param.y_range)  * 
                                        self.labels.apply.opts(xlim=self.base_rng.param.x_range,
                                                               ylim=self.base_rng.param.y_range) * 
                                        self.cov_map.apply.opts(visible=self.cov_viz.param.value, 
                                                                xlim=self.base_rng.param.x_range,
                                                                ylim=self.base_rng.param.y_range) *
                                        self.ndvi_map.apply.opts(visible=self.ndvi_viz.param.value, 
                                                                 xlim=self.base_rng.param.x_range,
                                                                 ylim=self.base_rng.param.y_range) *
                                        self.ndvi_rel_map.apply.opts(visible=self.ndvi_rel_viz.param.value, 
                                                                 xlim=self.base_rng.param.x_range,
                                                                 ylim=self.base_rng.param.y_range) *
                                       self.bm_rel_map.apply.opts(visible=self.bm_rel_viz.param.value, 
                                                                 xlim=self.base_rng.param.x_range,
                                                                 ylim=self.base_rng.param.y_range) *
                                       self.cp_rel_map.apply.opts(visible=self.cp_rel_viz.param.value, 
                                                                 xlim=self.base_rng.param.x_range,
                                                                 ylim=self.base_rng.param.y_range) *
                                        self.bm_map.apply.opts(visible=self.bm_viz.param.value, 
                                                               xlim=self.base_rng.param.x_range,
                                                               ylim=self.base_rng.param.y_range) *
                                        self.bare_chng_map.apply.opts(visible=self.bare_chng_viz.param.value, 
                                                               xlim=self.base_rng.param.x_range,
                                                               ylim=self.base_rng.param.y_range) *
                                        self.ndvi_chng_map.apply.opts(visible=self.ndvi_chng_viz.param.value, 
                                                               xlim=self.base_rng.param.x_range,
                                                               ylim=self.base_rng.param.y_range) *
                                        self.bm_chng_map.apply.opts(visible=self.bm_chng_viz.param.value, 
                                                               xlim=self.base_rng.param.x_range,
                                                               ylim=self.base_rng.param.y_range) *
                                        self.thresh_map.apply.opts(visible=self.thresh_viz.param.value, 
                                                                   xlim=self.base_rng.param.x_range,
                                                                   ylim=self.base_rng.param.y_range) *
                                       self.cp_map.apply.opts(visible=self.cp_viz.param.value, 
                                                               xlim=self.base_rng.param.x_range,
                                                               ylim=self.base_rng.param.y_range) *
                                       self.cp_chng_map.apply.opts(visible=self.cp_chng_viz.param.value, 
                                                               xlim=self.base_rng.param.x_range,
                                                               ylim=self.base_rng.param.y_range) *
                                        self.past_polys * 
                                        self.polys, 
                                      sizing_mode="stretch_both")
                return self.all_maps
            else:
                print('test')
                return pn.Column(self.tiles * self.labels)
        else:
            return pn.Column(self.tiles * self.labels)

    @pm.depends('year_picker.value', watch=True)
    def update_stats(self, data=None, index=None, value=None):
        if data is None and self.poly_stream.data is None:
            self.bm_gauge.value = 0
            self.cp_gauge.value = 0.0
            self.bm_ts_dat_poly = []
            self.ndvi_ts_dat_poly = []
            self.cp_ts_dat_poly = []
        elif data is None and self.poly_stream.data is not None:
            data = self.poly_stream.data
            self.bm_ts_dat_poly = []
            self.ndvi_ts_dat_poly = []  
            self.cp_ts_dat_poly = []
        if data is not None and self.poly_stream.data is not None:
            if len(data['xs']) == 0:
                self.bm_gauge.value = 0
                self.cp_gauge.value = 0.0
                self.bm_ts_dat_poly = []
                self.ndvi_ts_dat_poly = []
                self.cp_ts_dat_poly = []
            elif len(data['xs'][0]) < 3:
                self.bm_gauge.value = 0
                self.cp_gauge.value = 0.0
                self.bm_ts_dat_poly = []
                self.ndvi_ts_dat_poly = []
                self.cp_ts_dat_poly = []
                cov_pie_dict_poly = deepcopy(self.cov_echart.object)
                    
                cov_pie_dict_poly['series'][0]['data'] = [{'value': 0, 'name': 'Litter'},
                                 {'value': 0, 'name': 'Bare ground'},
                                 {'value': 0, 'name': 'Green veg'},
                                 {'value': 0, 'name': 'Dry veg'}]

                self.cov_echart.object = cov_pie_dict_poly
                
                thresh_bar_dict_poly = deepcopy(self.thresh_echart.object)
                thresh_bar_dict_poly['title']['subtext'] = ''
                thresh_bar_dict_poly['xAxis']['name'] = 'Probability of biomass less than ' + str(self.thresh_picker.value_throttled) + 'lbs/ac'
                thresh_bar_dict_poly['yAxis']['max'] = 1000
                thresh_bar_dict_poly['series'][0]['data'] = [0 for x in self.thresh_labels]
                self.thresh_echart.object = thresh_bar_dict_poly
                
            else:
                if gcloud:
                    self.bm_gauge.loading = True
                    self.cp_gauge.loading = True
                    self.bm_echart.loading = True
                    self.thresh_echart.loading = True
                    self.cov_echart.loading = True
                    self.ts_bm_echart.loading = True
                    self.ts_ndvi_echart.loading = True
                    self.ts_cp_echart.loading = True
                    self.ts_cov_echart_pasts.loading = True
                    self.ts_cov_echart_poly.loading = True
                polys_tmp = gpd.GeoDataFrame(data=data)
                polys_tmp.set_geometry(polys_tmp.apply(lambda row: Polygon(zip(row['xs'], row['ys'])), axis=1), inplace=True)
                polys_tmp.set_crs(epsg='3857', inplace=True)
                polys_info = polys_tmp[['line_color', 'geometry']].reset_index(drop=True).reset_index().rename(columns={'index': 'id'})
                polys_mask_shp = [(row.geometry, row.id+1) for _, row in polys_info.iterrows()]
                polys_mask = shp2mask(shp=polys_mask_shp, 
                                     transform=self.ds_sel['Biomass'].rio.transform(), 
                                     outshape=self.ds_sel['Biomass'].shape, 
                                     xr_object=self.ds_sel['Biomass'])
                poly_mask_tmp = polys_mask.where(polys_mask == 1, drop=True)
                bm_dat_tmp_poly = self.ds_sel['Biomass'].sel(x=poly_mask_tmp['x'],
                                                             y=poly_mask_tmp['y'],
                                                             method='nearest',
                                                             tolerance=30).where(poly_mask_tmp == 1)   
                cp_dat_tmp_poly = self.ds_sel['CP'].sel(x=poly_mask_tmp['x'],
                                                        y=poly_mask_tmp['y'],
                                                        method='nearest',
                                                        tolerance=30).where(poly_mask_tmp == 1)   
                
                
                if not bm_dat_tmp_poly.isnull().all():
                    # single-date variables
                    bm_hist_dat_poly = bm_dat_tmp_poly.to_dataset().groupby_bins('Biomass', bins=self.bm_plotrange, include_lowest=True,
                                    labels=self.bm_plotrange[1:]).count() / bm_dat_tmp_poly.size * 100
                    bm_hist_dat_poly = bm_hist_dat_poly.fillna(0)
                    self.bm_gauge.value = int(bm_dat_tmp_poly.mean().values)
                    self.cp_gauge.value = np.round(cp_dat_tmp_poly.mean().round(2).values, 2)

                    thresh_poly_dat_tmp = self.da_thresh.sel(x=poly_mask_tmp['x'], 
                                                             y=poly_mask_tmp['y'],
                                                             method='nearest',
                                                             tolerance=30).where(poly_mask_tmp == 1)
                    thresh_bar_dat_poly = thresh_poly_dat_tmp.to_dataset().groupby_bins('Threshold', bins=self.thresh_plotrange, include_lowest=True,
                                    labels=self.thresh_labels).count() * 0.90
                    thresh_bar_dat_poly = thresh_bar_dat_poly.fillna(0)

                    cov_dat_tmp_poly = self.ds_sel[['SD', 'GREEN', 'BARE', 'LITT']].sel(x=poly_mask_tmp['x'],
                                                                                        y=poly_mask_tmp['y'],
                                                                                        method='nearest',
                                                                                        tolerance=30).where(poly_mask_tmp == 1)
                    cov_labels_poly = ['Dry veg', 'Green veg', 'Bare ground', 'Litter']
                    cov_vals_poly = [int(round(float(cov_dat_tmp_poly[f].mean())*100, 0)) for f in cov_dat_tmp_poly.keys()]
        
                    bm_hist_dict_poly = deepcopy(self.bm_echart.object)
        
                    bm_hist_dict_poly['title']['subtext'] = str(bm_hist_dat_poly['Biomass'].cumsum()[
                                    bm_hist_dat_poly['Biomass_bins'] == self.thresh_picker.value_throttled].astype('int').values[0]) +\
            '% of the area is less than the threshold of ' + \
            str(self.thresh_picker.value_throttled) + ' lbs/ac'
                    bm_hist_dict_poly['yAxis']['max'] = round(max(bm_hist_dat_poly.Biomass.values)*1.35, 0)
                    bm_hist_dict_poly['series'] = [{
                            'name': "",
                            'type': "bar",
                            'data': [{'value': [bm_hist_dat_poly.Biomass_bins.values[idx], x], 
                                      'itemStyle': {'color': self.bm_colors[idx]}} for idx, x in enumerate(
                                np.round(bm_hist_dat_poly.Biomass.values, 1))],
                            'colorBy': "data",
                            'markLine': {
                                'silent': True,
                                'data': [
                                    [{'coord': [self.thresh_picker.value_throttled, 0],
                                      'lineStyle': {
                                          'color': 'black'}}, 
                                     {'coord': [self.thresh_picker.value_throttled, round(max(bm_hist_dat_poly.Biomass.values)*1.15, 0)],
                                      'symbol': 'none'}]],
                                'lineStyle': {'color': 'black'},
                                'label': {'formatter': str(self.thresh_picker.value_throttled) + ' lbs/ac', 
                                          'distance': 10,
                                          'color': 'black'}}}]
        
                    
                    self.bm_echart.object = bm_hist_dict_poly
                    
                    thresh_bar_dict_poly = deepcopy(self.thresh_echart.object)
                    thresh_bar_dict_poly['title']['subtext'] = 'Area in each category (' + str(int(bm_dat_tmp_poly.notnull().sum()*0.90)) + 'ha total)'
                    thresh_bar_dict_poly['xAxis']['name'] = 'Probability of biomass less than ' + str(self.thresh_picker.value_throttled) + 'lbs/ac'
                    thresh_bar_dict_poly['yAxis']['max'] = round(max(thresh_bar_dat_poly.Threshold.values)*1.10, -1)
                    thresh_bar_dict_poly['series'][0]['data'] = [{'value': x, 
                                                                   'itemStyle': {'color': self.thresh_colors[idx]}} for idx, x in enumerate(
                                np.round(thresh_bar_dat_poly.Threshold.values, -1))]
                    
                    self.thresh_echart.object = thresh_bar_dict_poly

                    cov_pie_dict_poly = deepcopy(self.cov_echart.object)
                    
                    cov_pie_dict_poly['series'][0]['data'] = [{'value': cov_vals_poly[0], 'name': cov_labels_poly[0]},
                                                              {'value': cov_vals_poly[1], 'name': cov_labels_poly[1]},
                                                              {'value': cov_vals_poly[2], 'name': cov_labels_poly[2]},
                                                              {'value': cov_vals_poly[3], 'name': cov_labels_poly[3]}]

                    self.cov_echart.object = cov_pie_dict_poly
                    
                
                ds_tmp_poly = self.ds_ts.sel(x=poly_mask_tmp['x'],
                                             y=poly_mask_tmp['y'],
                                             method='nearest', 
                                             tolerance=30).where(poly_mask_tmp == 1)
                
                self.bm_ts_dat_poly = list(map(list, zip(ds_tmp_poly.date.astype(str).values, 
                                                         ds_tmp_poly.mean(['y', 'x'])['Biomass'].astype('int').values)))
                self.ndvi_ts_dat_poly = list(map(list, zip(ds_tmp_poly.date.astype(str).values, 
                                                         ds_tmp_poly.mean(['y', 'x'])['NDVI'].values)))
                self.ndvi_cp_dat_poly = list(map(list, zip(ds_tmp_poly.date.astype(str).values, 
                                                         ds_tmp_poly.mean(['y', 'x'])['CP'].values)))
                self.cov_ts_dat_poly = {'Bare': list(map(list, zip(ds_tmp_poly.date.astype(str).values,
                                                                   np.round(ds_tmp_poly.mean(['y', 'x'])['BARE'].values * 100, 1)))),
                                        'Litter': list(map(list, zip(ds_tmp_poly.date.astype(str).values,
                                                                     np.round(ds_tmp_poly.mean(['y', 'x'])['LITT'].values * 100, 1)))),
                                        'Dry': list(map(list, zip(ds_tmp_poly.date.astype(str).values,
                                                                  np.round(ds_tmp_poly.mean(['y', 'x'])['SD'].values * 100, 1)))),
                                        'Green': list(map(list, zip(ds_tmp_poly.date.astype(str).values,
                                                                    np.round(ds_tmp_poly.mean(['y', 'x'])['GREEN'].values * 100, 1))))}

                ts_cov_dict_poly = deepcopy(self.ts_cov_echart_poly.object)

                for idx, k in enumerate(self.cov_ts_dat_poly):
                    ts_cov_dict_poly['series'][idx]['data'] = self.cov_ts_dat_poly[k]
                self.ts_cov_echart_poly.object = ts_cov_dict_poly
        
        if len(self.past_sel.index) == 0:
            self.bm_gauge_pasts.value = 0
            self.cp_gauge_pasts.value = 0
            self.bm_echart_pasts.object = self.bm_hist_pasts
            self.thresh_echart_pasts.object = self.thresh_bar_pasts
            self.cov_echart_pasts.object = self.cov_pie_pasts
            self.ts_bm_echart.object = self.ts_bm.copy()
            self.ts_ndvi_echart.object = self.ts_ndvi.copy()
            self.ts_cp_echart.object = self.ts_cp.copy()
            self.stats_title.title = 'CPER-wide'
            self.past_name = 'Pasture'
        elif index is None and len(self.past_sel.index) > 0:
            index = self.past_sel.index
        if index is not None and len(index) > 0:
            pasts_tmp = self.past_polys.data.iloc[index]
            self.past_name = self.past_polys.data['Pasture'][index].values[0]
            self.stats_title.title =self.past_name
            pasts_info = pasts_tmp[['linecolor', 'geometry']].reset_index(drop=True).reset_index().rename(columns={'index': 'id'})
            pasts_mask_shp = [(row.geometry, row.id+1) for _, row in pasts_info.iterrows()]
            pasts_mask = shp2mask(shp=pasts_mask_shp, 
                                 transform=self.ds_sel['Biomass'].rio.transform(), 
                                 outshape=self.ds_sel['Biomass'].shape, 
                                 xr_object=self.ds_sel['Biomass'])
            pasts_mask_tmp = pasts_mask.where(pasts_mask == 1, drop=True)
            bm_dat_tmp_pasts = self.ds_sel['Biomass'].sel(x=pasts_mask_tmp['x'],
                                                          y=pasts_mask_tmp['y'], 
                                                          method='nearest', 
                                                          tolerance=30).where(pasts_mask == 1)     
            cp_dat_tmp_pasts = self.ds_sel['CP'].sel(x=pasts_mask_tmp['x'],
                                                          y=pasts_mask_tmp['y'], 
                                                          method='nearest', 
                                                          tolerance=30).where(pasts_mask == 1)     
            
            if not bm_dat_tmp_pasts.isnull().all():
                # single-date variables
                bm_hist_dat_pasts = bm_dat_tmp_pasts.to_dataset().groupby_bins('Biomass', bins=self.bm_plotrange, include_lowest=True,
                                labels=self.bm_plotrange[1:]).count() / bm_dat_tmp_pasts.count() * 100
                bm_hist_dat_pasts = bm_hist_dat_pasts.fillna(0)
                self.bm_gauge_pasts.value = int(bm_dat_tmp_pasts.mean().values)
                self.cp_gauge_pasts.value = np.round(cp_dat_tmp_pasts.mean().round(2).values, 2)

                bm_hist_dict_pasts = deepcopy(self.bm_echart_pasts.object)
        
                bm_hist_dict_pasts['title']['subtext'] = str(bm_hist_dat_pasts['Biomass'].cumsum()[
                                bm_hist_dat_pasts['Biomass_bins'] == self.thresh_picker.value_throttled].astype('int').values[0]) +\
        '% of the area is less than the threshold of ' + \
        str(self.thresh_picker.value_throttled) + ' lbs/ac'
                bm_hist_dict_pasts['yAxis']['max'] = round(max(bm_hist_dat_pasts.Biomass.values)*1.35, 0)
                bm_hist_dict_pasts['series'] = [{
                        'name': "",
                        'type': "bar",
                        'data': [{'value': [bm_hist_dat_pasts.Biomass_bins.values[idx], x], 
                                  'itemStyle': {'color': self.bm_colors[idx]}} for idx, x in enumerate(
                            np.round(bm_hist_dat_pasts.Biomass.values, 1))],
                        'colorBy': "data",
                        'markLine': {
                            'silent': True,
                            'data': [
                                [{'coord': [self.thresh_picker.value_throttled, 0],
                                  'lineStyle': {
                                      'color': 'black'}}, 
                                 {'coord': [self.thresh_picker.value_throttled, round(max(bm_hist_dat_pasts.Biomass.values)*1.15, 0)],
                                  'symbol': 'none'}]],
                            'lineStyle': {'color': 'black'},
                            'label': {'formatter': str(self.thresh_picker.value_throttled) + ' lbs/ac', 
                                      'distance': 10,
                                      'color': 'black'}}}]


                self.bm_echart_pasts.object = bm_hist_dict_pasts
                
                
                thresh_pasts_dat_tmp = self.da_thresh.sel(x=pasts_mask_tmp['x'],
                                                          y=pasts_mask_tmp['y'],
                                                          method='nearest',
                                                          tolerance=30).where(pasts_mask == 1)
                thresh_bar_dat_pasts = thresh_pasts_dat_tmp.to_dataset().groupby_bins('Threshold', bins=self.thresh_plotrange, include_lowest=True,
                                labels=self.thresh_labels).count() * 0.090 * 2.47105
                thresh_bar_dat_pasts = thresh_bar_dat_pasts.fillna(0)

                cov_dat_tmp_pasts = self.ds_sel[['SD', 'GREEN', 'BARE', 'LITT']].sel(x=pasts_mask_tmp['x'], 
                                                                                     y=pasts_mask_tmp['y'],
                                                                                     method='nearest',
                                                                                     tolerance=30).where(pasts_mask == 1)
                cov_labels_pasts = ['Dry veg', 'Green veg', 'Bare ground', 'Litter']
                cov_vals_pasts = [int(round(float(cov_dat_tmp_pasts[f].mean())*100, 0)) for f in cov_dat_tmp_pasts.keys()]
                
                thresh_bar_dict_pasts = deepcopy(self.thresh_echart_pasts.object)
                thresh_bar_dict_pasts['title']['subtext'] = 'Area in each category (' + str(int(bm_dat_tmp_pasts.count()*0.090 * 2.47105)) + ' acres total)'
                thresh_bar_dict_pasts['xAxis']['name'] = 'Probability of biomass less than ' + str(self.thresh_picker.value_throttled) + 'lbs/ac'
                thresh_bar_dict_pasts['yAxis']['max'] = round(max(thresh_bar_dat_pasts.Threshold.values)*1.10, -1)
                thresh_bar_dict_pasts['series'][0]['data'] = [{'value': x, 
                                                               'itemStyle': {'color': self.thresh_colors[idx]}} for idx, x in enumerate(
                            np.round(thresh_bar_dat_pasts.Threshold.values, -1))]

                self.thresh_echart_pasts.object = thresh_bar_dict_pasts

                cov_pie_dict_pasts = deepcopy(self.cov_echart_pasts.object)

                cov_pie_dict_pasts['series'][0]['data'] = [{'value': cov_vals_pasts[0], 'name': cov_labels_pasts[0]},
                                                          {'value': cov_vals_pasts[1], 'name': cov_labels_pasts[1]},
                                                          {'value': cov_vals_pasts[2], 'name': cov_labels_pasts[2]},
                                                          {'value': cov_vals_pasts[3], 'name': cov_labels_pasts[3]}]

                self.cov_echart_pasts.object = cov_pie_dict_pasts
                

            self.bm_ts_dat_past = list(map(list, zip(aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                               (aoi_means['Year'] == str(self.ds_picker.value))]['date'],
                                                     aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                               (aoi_means['Year'] == str(self.ds_picker.value))]['Biomass'])))
            self.bm_ts_dat_past_raw = list(map(list, zip(aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                               (aoi_means['Year'] == str(self.ds_picker.value))]['date'],
                                                     aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                               (aoi_means['Year'] == str(self.ds_picker.value))]['Biomass_raw'])))
            self.bm_ts_dat_past_comp = list(map(list, zip(aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                               (aoi_means['Year'] == str(self.year_picker.value))]['date'],
                                                     aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                               (aoi_means['Year'] == str(self.year_picker.value))]['Biomass'])))
            self.bm_ts_dat_past_lta = list(map(list, zip(aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                  (aoi_means['Year'] == 'long-term avg.')]['date'],
                                        aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                  (aoi_means['Year'] == 'long-term avg.')]['Biomass'])))
            self.ndvi_ts_dat_past_lta = list(map(list, zip(aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                  (aoi_means['Year'] == 'long-term avg.')]['date'],
                                        aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                  (aoi_means['Year'] == 'long-term avg.')]['NDVI'])))
            self.ndvi_ts_dat_past = list(map(list, zip(aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                               (aoi_means['Year'] == str(self.ds_picker.value))]['date'],
                                                     aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                               (aoi_means['Year'] == str(self.ds_picker.value))]['NDVI'])))
            self.ndvi_ts_dat_past_raw = list(map(list, zip(aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                               (aoi_means['Year'] == str(self.ds_picker.value))]['date'],
                                                     aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                               (aoi_means['Year'] == str(self.ds_picker.value))]['NDVI_raw'])))
            self.ndvi_ts_dat_past_comp = list(map(list, zip(aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                            (aoi_means['Year'] == str(self.year_picker.value))]['date'],
                                                            aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                            (aoi_means['Year'] == str(self.year_picker.value))]['NDVI'])))
            self.cp_ts_dat_past_lta = list(map(list, zip(aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                         (aoi_means['Year'] == 'long-term avg.')]['date'],
                                                         aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                         (aoi_means['Year'] == 'long-term avg.')]['CP'])))
            self.cp_ts_dat_past = list(map(list, zip(aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                               (aoi_means['Year'] == str(self.ds_picker.value))]['date'],
                                                     aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                               (aoi_means['Year'] == str(self.ds_picker.value))]['CP'])))
            self.cp_ts_dat_past_comp = list(map(list, zip(aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                               (aoi_means['Year'] == str(self.year_picker.value))]['date'],
                                                     aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                               (aoi_means['Year'] == str(self.year_picker.value))]['CP'])))

            self.cov_ts_dat_pasts = {'Bare': list(map(list, zip(aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                                          (aoi_means['Year'] == str(self.ds_picker.value))]['date'],
                                                                aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                                          (aoi_means['Year'] == str(self.ds_picker.value))]['BARE']))),
                                    'Litter': list(map(list, zip(aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                                          (aoi_means['Year'] == str(self.ds_picker.value))]['date'],
                                                                aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                                          (aoi_means['Year'] == str(self.ds_picker.value))]['LITT']))),
                                    'Dry': list(map(list, zip(aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                                          (aoi_means['Year'] == str(self.ds_picker.value))]['date'],
                                                                aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                                          (aoi_means['Year'] == str(self.ds_picker.value))]['SD']))),
                                    'Green': list(map(list, zip(aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                                          (aoi_means['Year'] == str(self.ds_picker.value))]['date'],
                                                                aoi_means[(aoi_means['Pasture'] == self.past_name) & 
                                                                          (aoi_means['Year'] == str(self.ds_picker.value))]['GREEN'])))}

            ts_cov_dict_pasts = deepcopy(self.ts_cov_echart_pasts.object)

            for idx, k in enumerate(self.cov_ts_dat_pasts):
                ts_cov_dict_pasts['series'][idx]['data'] = self.cov_ts_dat_pasts[k]
            self.ts_cov_echart_pasts.object = ts_cov_dict_pasts

            # ADD ALL DATA TO BIOMASS TIME SERIES
            ts_bm_dict = deepcopy(self.ts_bm_echart.object)
            # drop the TRM and Heavy data
            ts_bm_dict['series'][5]['data'] = []
            ts_bm_dict['series'][6]['data'] = []
            # update all the pasture data
            ts_bm_dict['series'][0]['data'] = self.bm_ts_dat_past_lta
            ts_bm_dict['series'][1]['data'] = self.bm_ts_dat_past_comp
            ts_bm_dict['series'][1]['name'] = str(self.year_picker.value)
            ts_bm_dict['series'][1]['itemStyle'] = {'color': self.past_col}
            ts_bm_dict['series'][2]['data'] = self.bm_ts_dat_past
            ts_bm_dict['series'][2]['name'] = str(self.ds_picker.value)
            ts_bm_dict['series'][3]['data'] = self.bm_ts_dat_poly
            ts_bm_dict['series'][4]['data'] = self.bm_ts_dat_past_raw
            ts_bm_dict['legend']['data'] = [{'name': 'long-term avg.',
                                               'icon': 'path://M180 1000 l0 -30 200 0 200 0 0 30 0 30 -200 0 -200 0 0 -30z'},
                                            {'name': str(self.year_picker.value),
                                             'icon': 'path://M180 1000 l0 -40 200 0 200 0 0 40 0 40 -200 0 -200 0 0 -40z, M810 1000 l0 -40 200 0 200 0 0 40 0 40 -200 0 -200 0 0 -40zm, M1440 1000 l0 -40 200 0 200 0 0 40 0 40 -200 0 -200 0 0 -40z'},
                                            {'name': str(self.ds_picker.value),
                                            'icon': 'path://M180 1000 l0 -30 200 0 200 0 0 30 0 30 -200 0 -200 0 0 -30z'},
                                            {'name': 'Raw observation'},
                                            {'name': 'Drawing',
                                             'icon': 'path://M180 1000 l0 -30 200 0 200 0 0 30 0 30 -200 0 -200 0 0 -30z'}
                                           ]
            ts_bm_dict['legend']['data'] = [x for x in ts_bm_dict['legend']['data'] if x['name'] != 'None']
            if len(self.bm_ts_dat_poly) == 0:
                ts_bm_dict['legend']['data'] = [x for x in ts_bm_dict['legend']['data'] if x['name'] != 'Drawing']
            if np.all([np.isnan(x[1]) for x in self.bm_ts_dat_past_raw]):
                ts_bm_dict['legend']['data'] = [x for x in ts_bm_dict['legend']['data'] if x['name'] != 'Raw observation']
            self.ts_bm_echart.object = ts_bm_dict

            # ADD ALL DATA TO NDVI TIME SERIES
            ts_ndvi_dict = deepcopy(self.ts_ndvi_echart.object)
            # drop the TRM and Heavy data
            ts_ndvi_dict['series'][5]['data'] = []
            ts_ndvi_dict['series'][6]['data'] = []
            # update all the pasture data
            ts_ndvi_dict['series'][0]['data'] = self.ndvi_ts_dat_past_lta
            ts_ndvi_dict['series'][1]['data'] = self.ndvi_ts_dat_past_comp
            ts_ndvi_dict['series'][1]['name'] = str(self.year_picker.value)
            ts_ndvi_dict['series'][1]['itemStyle'] = {'color': self.past_col}
            ts_ndvi_dict['series'][2]['data'] = self.ndvi_ts_dat_past
            ts_ndvi_dict['series'][2]['name'] = str(self.ds_picker.value)
            ts_ndvi_dict['series'][3]['data'] = self.ndvi_ts_dat_poly
            ts_ndvi_dict['series'][4]['data'] = self.ndvi_ts_dat_past_raw
            ts_ndvi_dict['legend']['data'] = [{'name': 'long-term avg.',
                                               'icon': 'path://M180 1000 l0 -30 200 0 200 0 0 30 0 30 -200 0 -200 0 0 -30z'},
                                            {'name': str(self.year_picker.value),
                                              'icon': 'path://M180 1000 l0 -40 200 0 200 0 0 40 0 40 -200 0 -200 0 0 -40z, M810 1000 l0 -40 200 0 200 0 0 40 0 40 -200 0 -200 0 0 -40zm, M1440 1000 l0 -40 200 0 200 0 0 40 0 40 -200 0 -200 0 0 -40z'},
                                            {'name': str(self.ds_picker.value),
                                             'icon': 'path://M180 1000 l0 -30 200 0 200 0 0 30 0 30 -200 0 -200 0 0 -30z'},
                                              {'name': 'Raw observation'},
                                            {'name': 'Drawing',
                                             'icon': 'path://M180 1000 l0 -30 200 0 200 0 0 30 0 30 -200 0 -200 0 0 -30z'}
                                             ]
            ts_ndvi_dict['legend']['data'] = [x for x in ts_ndvi_dict['legend']['data'] if x['name'] != 'None']
            if len(self.ndvi_ts_dat_poly) == 0:
                ts_ndvi_dict['legend']['data'] = [x for x in ts_ndvi_dict['legend']['data'] if x['name'] != 'Drawing']
            if np.all([np.isnan(x[1]) for x in self.ndvi_ts_dat_past_raw]):
                ts_ndvi_dict['legend']['data'] = [x for x in ts_ndvi_dict['legend']['data'] if x['name'] != 'Raw observation']
            self.ts_ndvi_echart.object = ts_ndvi_dict

            # ADD ALL DATA TO CRUDE PROTEIN TIME SERIES
            ts_cp_dict = deepcopy(self.ts_cp_echart.object)
            # drop the TRM and Heavy data
            ts_cp_dict['series'][4]['data'] = []
            ts_cp_dict['series'][5]['data'] = []
            # update all the pasture data
            ts_cp_dict['series'][0]['data'] = self.cp_ts_dat_past_lta
            ts_cp_dict['series'][1]['data'] = self.cp_ts_dat_past_comp
            ts_cp_dict['series'][1]['name'] = str(self.year_picker.value)
            ts_cp_dict['series'][1]['itemStyle'] = {'color': self.past_col}
            ts_cp_dict['series'][2]['data'] = self.cp_ts_dat_past
            ts_cp_dict['series'][2]['name'] = str(self.ds_picker.value)
            ts_cp_dict['series'][3]['data'] = self.cp_ts_dat_poly
            ts_cp_dict['legend']['data'] = [{'name': 'long-term avg.',
                                               'icon': 'path://M180 1000 l0 -30 200 0 200 0 0 30 0 30 -200 0 -200 0 0 -30z'},
                                            {'name': str(self.year_picker.value),
                                              'icon': 'path://M180 1000 l0 -40 200 0 200 0 0 40 0 40 -200 0 -200 0 0 -40z, M810 1000 l0 -40 200 0 200 0 0 40 0 40 -200 0 -200 0 0 -40zm, M1440 1000 l0 -40 200 0 200 0 0 40 0 40 -200 0 -200 0 0 -40z'},
                                            {'name': str(self.ds_picker.value),
                                             'icon': 'path://M180 1000 l0 -30 200 0 200 0 0 30 0 30 -200 0 -200 0 0 -30z'},
                                            {'name': 'Drawing',
                                             'icon': 'path://M180 1000 l0 -30 200 0 200 0 0 30 0 30 -200 0 -200 0 0 -30z'}
                                             ]
            ts_cp_dict['legend']['data'] = [x for x in ts_cp_dict['legend']['data'] if x['name'] != 'None']
            if len(self.cp_ts_dat_poly) == 0:
                ts_cp_dict['legend']['data'] = [x for x in ts_cp_dict['legend']['data'] if x['name'] != 'Drawing']
            self.ts_cp_echart.object = ts_cp_dict
        
        if gcloud:
            self.bm_gauge.loading = False
            self.cp_gauge.loading = False
            self.bm_echart.loading = False
            self.thresh_echart.loading = False
            self.cov_echart.loading = False
            self.bm_gauge_pasts.loading = False
            self.cp_gauge_pasts.loading = False
            self.bm_echart_pasts.loading = False
            self.thresh_echart_pasts.loading = False
            self.cov_echart_pasts.loading = False
            self.ts_bm_echart.loading = False
            self.ts_ndvi_echart.loading = False
            self.ts_cp_echart.loading = False
            self.ts_cov_echart_pasts.loading = False
            self.ts_cov_echart_poly.loading = False

    @pm.depends('thresh_picker.value_throttled', watch=True)
    def trigger_thresh(self):
        if not self.poly_stream.data is None:
            if len(self.poly_stream.data['xs']) > 0:
                self.update_stats(data=self.poly_stream.data)
    
    @pm.depends('viz_sel.value')
    def update_colorbar(self):
        if self.viz_sel.value == 'Basemap':
            return pn.Spacer(height=20, width=self.control_width)
        elif self.viz_sel.value == 'Biomass threshold':
            return self.make_colorbar(self.viz_sel.value).options(color_levels=5)
        else:
            return self.make_colorbar(self.viz_sel.value)
        
    @pm.depends('viz_sel.value')
    def update_slider(self):
        if self.viz_sel.value in ['Biomass change', 'Greenness change', 'Bare ground change', 'Crude protein change']:
            return self.date_diff_picker
        elif self.viz_sel.value == 'Biomass threshold':
            return self.thresh_picker
        else:
            return pn.Spacer(height=20, width=self.control_width)


    @pm.depends('stats_title.title')
    def update_stats_title(self):
        if self.stats_title.title == 'CPER-wide':
            return pn.pane.HTML(
                '<div style="text-align:center">' + 
                '<h1>No pasture selected</h1>'
                '</div>',
                width=self.stats_width)
        else:
            return pn.pane.HTML(
                '<div style="text-align:center">' + 
                '<h1>' + self.stats_title.title + '</h1>'
                '</div>',
                width=self.stats_width)
        
    @pm.depends('stats_title.title')
    def update_ts_stats_title(self):
        return pn.pane.HTML(
            '<div style="text-align:center">' + 
            '<h1>' + self.stats_title.title + '</h1>'
            '</div>',
        width=self.stats_width)
        
    
    def _create_view(self):
        layout = pn.Column(pn.Row(pn.Column(pn.Column(pn.pane.Markdown('## Map options'),
                                                      self.ds_picker,
                                                      self.date, 
                                                      self.controls,
                                                      height=800,
                                                      scroll=False,
                                                      width=int(self.control_width*1.3),
                                                      #sizing_mode='scale_width',
                                                      css_classes=['panel-widget-box']),
                                            pn.Spacer(height=5, width=int(self.control_width*1.3)),
                                            pn.Column(pn.pane.Markdown('## Chart options'),
                                                      self.year_picker,
                                                      width=int(self.control_width*1.3),
                                                      #sizing_mode='scale_width',
                                                      css_classes=['panel-widget-box']),
                                            max_width=int(self.control_width*1.3),
                                            sizing_mode='scale_width'), 
                                  pn.Column(self.create_maps, sizing_mode='stretch_both'),
                                  pn.Tabs(('Pasture stats',
                                           pn.Column(
                                               self.update_stats_title,
                                               self.bm_gauge_pasts,
                                               self.cp_gauge_pasts,
                                               self.cov_echart_pasts,
                                               self.bm_echart_pasts, 
                                               self.thresh_echart_pasts, 
                                               scroll=True,
                                               max_height=800, 
                                               min_width=int(self.stats_width*1.1),
                                               sizing_mode='scale_height')),
                                          ('Time-series',
                                           pn.Column(
                                               self.update_ts_stats_title,
                                               self.ts_bm_echart,
                                               self.ts_ndvi_echart,
                                               self.ts_cp_echart,
                                               self.ts_cov_echart_pasts,
                                               self.ts_cov_echart_poly, 
                                               scroll=True,
                                               max_height=800, 
                                               min_width=int(self.stats_width*1.1), 
                                               sizing_mode='scale_height',
                                               align='center')),
                                          ('Drawing stats', 
                                           pn.Column(
                                               pn.pane.HTML(
                                                   '<div style="text-align:center">' + 
                                                   '<h1>Drawing</h1>',
                                                   width=self.stats_width),
                                               self.bm_gauge,
                                               self.cp_gauge,
                                               self.cov_echart,
                                               self.bm_echart, 
                                               self.thresh_echart, 
                                               scroll=True,
                                               max_height=800,
                                               min_width=int(self.stats_width*1.1), 
                                               sizing_mode='scale_height')),
                                         )))
        return layout
                        

In [ ]:
print('launching app')
app = getData()
if not gcloud and browser:
    pn.serve(app.view.servable())
else:
    app.view.servable()
